In [9]:
import os

from dotenv import load_dotenv
_ = load_dotenv()

# ensure the environment variable is set
print("OPENAI_API_KEY is loaded from .env file:", os.getenv("OPENAI_API_KEY"))


OPENAI_API_KEY is loaded from .env file: 


In [10]:
# install LangGraph and langchain

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [11]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json
